In [1]:
#i'm taking this ipynb file to experiment my  code bit by bit before running the whole code once

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [3]:
load_dotenv()

True

In [4]:
key = os.getenv("OPENAI_API_KEY")

In [5]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.7)

c:\Users\sumen\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
with open("response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [8]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [9]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [12]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],output_variables=["quiz", "review"], verbose=True,)

In [14]:

with open("data.txt","r") as file:
    TEXT=file.read()

In [15]:
NUMBER=5
SUBJECT="AI"
TONE="simple"

In [16]:
with  get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {

        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)

        }
    )

c:\Users\sumen\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize 
the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle 
of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, 
applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. 
We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of 
interest include but are not limited to:
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 5 multiple choice questions for AI s

In [17]:
response

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize \nthe way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle \nof building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, \napplications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. \nWe invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of \ninterest include but are not limited to:',
 'number': 5,
 'subject': 'AI',
 'tone': 'simple',
 'RESPONSE_JSON': '{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"

In [18]:
quiz=response.get("quiz")

In [20]:
quiz=json.loads(quiz)

In [21]:
quiz

{'1': {'no': '1',
  'mcq': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'Building robots',
   'b': 'Developing mobile apps',
   'c': 'Creating Generative AI systems',
   'd': 'Designing websites'},
  'correct': 'c'},
 '2': {'no': '2',
  'mcq': 'What is one of the topics of interest for the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'Cooking recipes',
   'b': 'Ethics in technology',
   'c': 'Fashion trends',
   'd': 'Sports analytics'},
  'correct': 'b'},
 '3': {'no': '3',
  'mcq': 'What can generative models produce, according to the text?',
  'options': {'a': 'Realistic text, images, and video',
   'b': 'Music albums',
   'c': 'Cooking shows',
   'd': 'Fashion magazines'},
  'correct': 'a'},
 '4': {'no': '4',
  'mcq': 'What concerns are associated with Generative AI technology?',
  'options': {'a': 'Fairness, transparency, and accountability',
   'b': 'Speed, accuracy, and efficiency',
   'c': 'Color, size, and shape',
  

In [22]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [23]:
quiz=pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv",index=False)

In [4]:
from pathlib import Path
import json
Path.cwd()

WindowsPath('c:/Users/sumen/OneDrive/Desktop/LLM_apps/MCQ_test_using_llms')

In [5]:
with open(str(Path.cwd())+"\\response.json","r") as f:
    RESPONSE_JSON=json.load(f)
RESPONSE_JSON 

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}